In [1]:
import cv2
import numpy as np
import os

In [2]:
def load_and_preprocess_images(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        for img_file in os.listdir(os.path.join(data_dir, label)):
            img_path = os.path.join(data_dir, label, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize to 224x224
            img = img / 255.0  # Normalize to [0, 1]
            images.append(img)
            labels.append(label)
        return np.array(images), np.array(labels)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

ImportError: DLL load failed while importing _pywrap_tf2: A dynamic link library (DLL) initialization routine failed.

In [ ]:
def create_model(num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
# Load and preprocess images
images, labels = load_and_preprocess_images('C:\Users\Ganesh\PlantVillage-Dataset-master\PlantVillage-Dataset-master')
# Convert labels to categorical
labels = to_categorical(labels)

In [ ]:
# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Create and compile the model
model = create_model(num_classes=len(np.unique(labels)))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])   

In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')   

In [ ]:
import streamlit as st
from tensorflow.keras.preprocessing import image

In [ ]:
st.title("Plant Disease Prediction")

uploaded_file = st.file_uploader("Choose a leaf image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    img = image.load_img(uploaded_file, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)

    st.image(img, caption='Uploaded Image', use_column_width=True)
    st.write(f'Predicted Class: {predicted_class}')
   